# Import Demand

In [75]:
def intervalo_horario(horario):
    return '07:00' < horario < '09:00'
def intervalo_horario_am_pm(horario):
    return '07:00:00 AM' < horario < '09:00:00 AM'

### Read jsons

In [1]:
import os
import json

In [7]:
files = []
dir_path = './data/disciplinas/json/'
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(path)

In [8]:
len(files)

5205

In [9]:
disciplinas = []
for file in files:
    with open(dir_path + file, 'r+') as f:
        content = json.loads(f.read())
        disciplinas.append(content)

In [10]:
disciplinas[1000]

{'unidade': 'Escola Politécnica',
 'departamento': 'Eng Telecomunicações e Controle',
 'campus': 'São Paulo',
 'codigo': 'PTC3502',
 'nome': 'Estágio Supervisionado',
 'creditos_aula': 1,
 'creditos_trabalho': 6,
 'objetivos': 'Realização de um estágio relacionado ao futuro exercício profissional, com a supervisão de um professor do curso.',
 'programa_resumido': 'Estágio em empresa, universidade ou instituição de pesquisa.',
 'turmas': [{'codigo': '2022250',
   'inicio': '15/08/2022',
   'fim': '21/12/2022',
   'tipo': 'Teórica',
   'horario': [{'dia': 'sab',
     'inicio': '09:20',
     'fim': '10:10',
     'professores': ['Cristiano Magalhaes Panazio']}],
   'vagas': {'Obrigatória': {'vagas': 35,
     'inscritos': 8,
     'pendentes': 0,
     'matriculados': 8,
     'grupos': {'EP - Engenharia Elétrica - EC3 (Telecomunicações)': {'vagas': 35,
       'inscritos': 6,
       'pendentes': 0,
       'matriculados': 6}}},
    'Extracurricular': {'vagas': 2,
     'inscritos': 0,
     'pend

### Filter Cuaso

In [11]:
fora_cuaso = ['Escola de Artes, Ciências e Humanidades', 'Escola de Enfermagem', 'Faculdade de Direito', 'Faculdade de Medicina',
                'Faculdade de Saúde Pública', 'Faculdade de Saúde Pública e Faculdade de Ciências Farmacêuticas']
def is_disciplina_cuaso(disciplina, fora_cuaso):
    if disciplina['campus'] != 'São Paulo':
        return False
    if disciplina['unidade'] in fora_cuaso:
        return False
    return True

In [12]:
disciplinas_cuaso = []
for disciplina in disciplinas:
    if is_disciplina_cuaso(disciplina, fora_cuaso):
        disciplinas_cuaso.append(disciplina)
len(disciplinas_cuaso)

2390

### Get Classes

In [13]:
aulas = []
for i, disciplina in enumerate(disciplinas_cuaso):
    for turma in disciplina['turmas']:
        total_matriculados = 0
        for k, vaga in turma['vagas'].items():
            total_matriculados += vaga['matriculados']
            
        if turma['horario'] != None:
            for horarios in turma['horario']:
                aula = {}
                aula['indice_disc'] = i
                aula['codigo'] = disciplina['codigo']
                aula['codigo_turma'] = turma['codigo']
                aula['unidade'] = disciplina['unidade']
                aula['dia'] = horarios['dia']
                aula['horario_inicio'] = horarios['inicio']
                aula['total_matriculados'] = total_matriculados
                aulas.append(aula)

### Filter Classes

In [72]:
aulas_intervalo_horario = []
for aula in aulas:
    if intervalo_horario(aula['horario_inicio']):
        aulas_intervalo_horario.append(aula)

In [15]:
agrupamento_dia_instituto = {}
agrupamento_dia_instituto['seg'] = {}
agrupamento_dia_instituto['ter'] = {}
agrupamento_dia_instituto['qua'] = {}
agrupamento_dia_instituto['qui'] = {}
agrupamento_dia_instituto['sex'] = {}
agrupamento_dia_instituto['sab'] = {}
agrupamento_dia_instituto['dom'] = {}
for aula in aulas_intervalo_horario:
    if aula['unidade'] not in agrupamento_dia_instituto[aula['dia']]:
        agrupamento_dia_instituto[aula['dia']][aula['unidade']] = 0
    agrupamento_dia_instituto[aula['dia']][aula['unidade']] += aula['total_matriculados']

In [156]:
def encode(s):
    return s.encode('latin1').decode('unicode_escape')

with open('data/disciplinas/map_sem_unid_alunos.json', 'w+') as f:
    f.write(encode(json.dumps(agrupamento_dia_instituto, indent=4)))

### Read research

In [55]:
import pandas as pd
import datetime as dt

In [59]:
pesquisa_perguntas = pd.read_csv('./data/disciplinas/resposta_pesq.csv', delimiter=';')
pesquisa_perguntas['timestamp'] = list(map(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), pesquisa_perguntas['Carimbo de data/hora']))
pesquisa_perguntas = pesquisa_perguntas[pesquisa_perguntas['timestamp'] >= dt.datetime(2022, 10, 7, 0, 0, 0)]
len(pesquisa_perguntas)

4395

In [60]:
pesquisa = pd.DataFrame()
pesquisa['vinculo'] = pesquisa_perguntas['Qual o seu vínculo com a USP?']
pesquisa['instituto'] = pesquisa_perguntas['A qual instituto você está associado?']


# ida
pesquisa['utiliza_circular_ida'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para entrar na USP?'] >= 2
pesquisa['linhas_ida'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?']
pesquisa['ponto_embarque_ida'] = pesquisa_perguntas['Por qual ponto você normalmente embarca?']
pesquisa['ponto_desembarque_ida'] = pesquisa_perguntas['Em qual ponto de ônibus você normalmente desembarca?']
pesquisa['horario_ida'] = pesquisa_perguntas['Que horas você normalmente chega na USP?']


# volta
pesquisa['utiliza_circular_volta'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para sair da USP?'] >= 2
pesquisa['linhas_volta'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?.1']
pesquisa['ponto_embarque_volta'] = pesquisa_perguntas['Em qual o ponto de ônibus você embarca para sair da USP?']
pesquisa['ponto_desembarque_volta'] = pesquisa_perguntas['Por qual ponto você normalmente desembarca?']
pesquisa['horario_volta'] = pesquisa_perguntas['Que horas você normalmente chega no ponto de ônibus para sair da USP?']

pesquisa.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


### Filter students

In [92]:
pesquisa[['vinculo']].value_counts()

vinculo                                                        
Aluno de graduação                                                 2880
Aluno de pós-graduação                                              752
Funcionário                                                         644
Docente                                                              36
docente                                                              22
Professor                                                            21
professor                                                             8
Ex-aluno                                                              5
Professor                                                             5
Docente                                                               4
Professor titular                                                     1
aprimoramento                                                         1
Pós-doutorando                                                        1


In [70]:
def grad_ou_pos(x):
    return x == 'Aluno de graduação' or x == 'Aluno de pós-graduação'
pesquisa_alunos = pesquisa[list(map(grad_ou_pos, pesquisa['vinculo']))]
pesquisa_alunos[['vinculo']].value_counts()

vinculo               
Aluno de graduação        2880
Aluno de pós-graduação     752
dtype: int64

### Filtro horário e cálculo porcentagem carro por instituto

In [91]:
pesquisa_alunos.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


In [150]:
pesquisa_ida = pesquisa_alunos[pesquisa_alunos['horario_ida'].notna()]
pesquisa_alunos_intervalo = pesquisa_ida[list(map(intervalo_horario_am_pm, pesquisa_ida['horario_ida']))]
pesquisa_alunos_intervalo

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
142,Aluno de graduação,Escola Politécnica (Poli),True,"8022, 8032",Estação Butantã,Poli Metalurgia,7:40:00 AM,True,"8012, 8032",Poli Mecânica,Estação Butantã,3:00:00 PM
143,Aluno de graduação,Escola Politécnica (Poli),True,"8012, 8022, 8032",Estação Butantã,Poli Biênio,8:30:00 AM,True,"8012, 8032",Poli Biênio,Estação Butantã,5:40:00 PM
...,...,...,...,...,...,...,...,...,...,...,...,...
4524,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,"8012, 8032",Estação Butantã,Biblioteca Brasiliana,8:00:00 AM,True,"8012, 8032","R. Prof. Mello Moraes, 1473 (localizado entre ...",Estação Butantã,1:10:00 PM
4525,Aluno de graduação,Escola Politécnica (Poli),True,"8012, 8022, 8032",Estação Butantã,Poli Civil,7:30:00 AM,True,"8012, 8022, 8032",Poli Civil,Estação Butantã,5:00:00 PM
4527,Aluno de graduação,Instituto de Biociências (IB),True,"8022, 8032",Estação Butantã,Biociências,7:40:00 AM,True,"8022, 8032",Ponto Paços das Artes,Estação Butantã,10:15:00 PM
4528,Aluno de graduação,Escola de Comunicações e Artes (ECA),True,"8012, 8032",Estação Butantã,CRUSP (Praça da Reitoria),8:10:00 AM,True,8012,ECA,Estação Butantã,6:00:00 PM


In [151]:
porc_circular_por_instituto = pesquisa_alunos_intervalo[['instituto','utiliza_circular_ida']].groupby('instituto').agg('mean').reset_index()

pd.set_option('display.max_rows', len(porc_circular_por_instituto))
porc_circular_por_instituto

,instituto,utiliza_circular_ida
0,Curso de Ciências Moleculares (CCM),1.000000
1,ECA e MAC USP,1.000000
2,Escola Politécnica (Poli),0.945848
3,Escola de Aplicação da Faculdade de Educação (...,1.000000
4,"Escola de Artes, Ciências e Humanidades (EACH)",0.684211
5,Escola de Comunicações e Artes (ECA),0.948905
6,Escola de Educação Física e Esporte (EEFE),0.941176
7,Escola de Enfermagem (EE),1.000000
8,FFLCH,1.000000
9,Faculadae de Arquitetura e Urbanismo (FAU),1.000000


In [157]:
# TODO: de (nome instituto pesquisa) para (nome instituto matrusp)

In [153]:
porc_circular_por_instituto['instituto']

0                   Curso de Ciências Moleculares (CCM)
1                                         ECA e MAC USP
2                             Escola Politécnica (Poli)
3     Escola de Aplicação da Faculdade de Educação (...
4        Escola de Artes, Ciências e Humanidades (EACH)
5                  Escola de Comunicações e Artes (ECA)
6            Escola de Educação Física e Esporte (EEFE)
7                             Escola de Enfermagem (EE)
8                                                 FFLCH
9            Faculadae de Arquitetura e Urbanismo (FAU)
10            Faculdade de Ciências Farmacêuticas (FCF)
11                            Faculdade de Direito (FD)
12    Faculdade de Economia, Administração e Contabi...
13                           Faculdade de Educação (FE)
14    Faculdade de Filosofia, Letras e Ciências Huma...
15                           Faculdade de Medicina (FM)
16    Faculdade de Medicina Veterinária e Zootecnia ...
17                        Faculdade de Odontolog

### Filter: utiliza circular, butantã vs. p3, horário

In [125]:
def is_butanta(x): 
    return x == 'Estação Butantã' or x == 'PTREM'

intervalo_butanta = pesquisa_alunos[pesquisa_alunos['utiliza_circular_ida']]
intervalo_butanta = intervalo_butanta[list(map(intervalo_horario_am_pm, intervalo_butanta['horario_ida']))]
intervalo_butanta = intervalo_butanta[list(map(is_butanta, intervalo_butanta['ponto_embarque_ida']))]
intervalo_butanta

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
142,Aluno de graduação,Escola Politécnica (Poli),True,"8022, 8032",Estação Butantã,Poli Metalurgia,7:40:00 AM,True,"8012, 8032",Poli Mecânica,Estação Butantã,3:00:00 PM
143,Aluno de graduação,Escola Politécnica (Poli),True,"8012, 8022, 8032",Estação Butantã,Poli Biênio,8:30:00 AM,True,"8012, 8032",Poli Biênio,Estação Butantã,5:40:00 PM
145,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,8012,Estação Butantã,FEA,8:15:00 AM,True,8012,Física,Estação Butantã,11:50:00 AM
...,...,...,...,...,...,...,...,...,...,...,...,...
4520,Aluno de pós-graduação,Museu de Arqueologia e Etnologia (MAE),True,8022,Estação Butantã,Prof Almeida Prado,8:50:00 AM,True,"8012, 8022, 8032",depende,Estação Butantã,3:00:00 PM
4524,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,"8012, 8032",Estação Butantã,Biblioteca Brasiliana,8:00:00 AM,True,"8012, 8032","R. Prof. Mello Moraes, 1473 (localizado entre ...",Estação Butantã,1:10:00 PM
4525,Aluno de graduação,Escola Politécnica (Poli),True,"8012, 8022, 8032",Estação Butantã,Poli Civil,7:30:00 AM,True,"8012, 8022, 8032",Poli Civil,Estação Butantã,5:00:00 PM
4527,Aluno de graduação,Instituto de Biociências (IB),True,"8022, 8032",Estação Butantã,Biociências,7:40:00 AM,True,"8022, 8032",Ponto Paços das Artes,Estação Butantã,10:15:00 PM


### Transform to stations

TODO:
- pegar uma linha randomica
- linhas_ida e ponto_desembarque_ida

### Add employees and professors